In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("dataset_without.csv")

In [6]:
df=df.drop(columns=['Unnamed: 0'])

In [8]:
# Convert float rating to int class label (0-indexed for BERT classification)
df['label'] = df['Rating'].astype(int) - 1  # Ratings 1–5 -> Labels 0–4

# Remove original Rating column
df = df[['Review', 'label']].rename(columns={'Review': 'text'})

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [9]:
from transformers import AutoTokenizer

#model_name = "microsoft/deberta-v3-small"
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/119640 [00:00<?, ? examples/s]

Map:   0%|          | 0/29911 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

training_args = TrainingArguments(
    output_dir="./bert-review-classifier3",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",
    report_to="none",
    fp16=True
)


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



/tmp/ipython-input-3333095094.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,1.405200
100,1.158400
150,1.092900
200,1.081600
250,1.065000
300,1.109300
350,1.084800
400,1.052800
450,1.050900
500,1.015400


TrainOutput(global_step=29912, training_loss=0.652033192281195, metrics={'train_runtime': 2864.1881, 'train_samples_per_second': 167.084, 'train_steps_per_second': 10.443, 'total_flos': 1.573972728219648e+16, 'train_loss': 0.652033192281195, 'epoch': 4.0})

In [13]:
trainer.save_model('./epoch 4')

In [14]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print(" Training Accuracy after Epoch 4:", train_metrics["eval_accuracy"])
print(" Training F1 after Epoch 4:", train_metrics["eval_f1"])

 Training Accuracy after Epoch 4: 0.9333166165162153
 Training F1 after Epoch 4: 0.9329783826615042


In [15]:
eval_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(" Eval Accuracy after Epoch 4:", eval_metrics["eval_accuracy"])
print(" Eval F1 after Epoch 4:", eval_metrics["eval_f1"])

 Eval Accuracy after Epoch 4: 0.641603423489686
 Eval F1 after Epoch 4: 0.6423630188882729


In [16]:
model = AutoModelForSequenceClassification.from_pretrained('./epoch 4')

In [17]:

training_args = TrainingArguments(
    output_dir='./bert-review-classifier3',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",

    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Resume training from checkpoint
trainer.train(resume_from_checkpoint=True)


/tmp/ipython-input-1379963534.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
29950,0.283200
30000,0.292900
30050,0.397600
30100,0.347800
30150,0.322100
30200,0.304700
30250,0.342500
30300,0.350100
30350,0.302700
30400,0.293600


TrainOutput(global_step=44868, training_loss=0.09655803009991214, metrics={'train_runtime': 1426.6549, 'train_samples_per_second': 503.163, 'train_steps_per_second': 31.45, 'total_flos': 2.360959092329472e+16, 'train_loss': 0.09655803009991214, 'epoch': 6.0})

In [18]:
trainer.save_model('./epoch 6')

In [19]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print(" Training Accuracy after Epoch 6:", train_metrics["eval_accuracy"])
print(" Training F1 after Epoch 6:", train_metrics["eval_f1"])

 Training Accuracy after Epoch 6: 0.9708040789033768
 Training F1 after Epoch 6: 0.9706637413727799


In [20]:
eval_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(" Eval Accuracy after Epoch 6:", eval_metrics["eval_accuracy"])
print(" Eval F1 after Epoch 6:", eval_metrics["eval_f1"])

 Eval Accuracy after Epoch 6: 0.6340476747684798
 Eval F1 after Epoch 6: 0.6351128792302885


**Testing on whole  dataset**

In [21]:
# testing on whole balanced dataset
new_df = pd.read_csv("dataset_without.csv")

In [28]:
new_df['label'] = new_df['Rating'].astype(int) - 1
new_df = new_df[['Review', 'label']].rename(columns={'Review': 'text'})

In [29]:
new_dataset = Dataset.from_pandas(new_df)

In [31]:
tokenizer = AutoTokenizer.from_pretrained('./epoch 6')

In [32]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

new_dataset = new_dataset.map(tokenize, batched=True)
new_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/149551 [00:00<?, ? examples/s]

In [33]:
model = AutoModelForSequenceClassification.from_pretrained('./epoch 6')

In [34]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }


from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    do_train=False,
    do_eval=True,
    report_to="none",
    fp16=True
)

In [35]:

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-678010718.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [36]:
from sklearn.metrics import classification_report
target_names = ["Rating 1", "Rating 2", "Rating 3", "Rating 4", "Rating 5"]
predictions_output = trainer.predict(new_dataset)
y_pred = np.argmax(predictions_output.predictions, axis=1)
y_true = predictions_output.label_ids

print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4,target_names=target_names))



Classification Report:
              precision    recall  f1-score   support

    Rating 1     0.9260    0.9362    0.9311     27888
    Rating 2     0.8302    0.8070    0.8184     14823
    Rating 3     0.8633    0.8578    0.8606     21670
    Rating 4     0.8917    0.8924    0.8920     38052
    Rating 5     0.9401    0.9443    0.9422     47118

    accuracy                         0.9035    149551
   macro avg     0.8902    0.8875    0.8889    149551
weighted avg     0.9031    0.9035    0.9033    149551



In [38]:
import torch

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Example review text
texts = [
    "not awesome ",

]

# Tokenize
inputs = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
).to(device)

# Prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)+1

print("Predicted labels:", predictions.cpu().numpy())

Predicted labels: [2]
